# Groq + Browser Use MCP: Intelligent Product Comparison
**The Problem:** Comparing products online means opening multiple tabs, checking various retailers, and manually tracking prices and features across different sources.

**The Solution:** This tutorial shows how to combine Groq's fast inference with Browser Use's Model Context Protocol (MCP) server to automatically research and compare products across multiple websites in real-time.

## What is Browser Use MCP?
Browser Use MCP enables AI models to autonomously browse websites, extract information, and interact with web pages. Think of it as giving your model the ability to:
- Navigate websites and click links
- Extract data from multiple sources
- Compare information across retailers

You can learn more about Browser Use at [browser-use.com](https://browser-use.com).

## Prerequisites

In [12]:
# install dependencies
!pip install groq python-dotenv

## Setup and Configuration

In [13]:
#!/usr/bin/env python3
"""
Groq + Browser Use MCP Demo
"""

import json
import os
import time
from datetime import datetime

from groq import Groq

# Set API Keys

Here we will set the API keys for the Groq and Browser Use APIs via either environment variables or a userdata object in Google Colab.

- You can get your Groq API key from [here](https://console.groq.com/keys)
- You can get your Browser Use API key from [here](https://browser-use.com)

In [14]:
try:
    from google.colab import userdata

    GROQ_API_KEY = userdata.get("GROQ_API_KEY")
    BROWSER_USE_API_KEY = userdata.get("BROWSER_USE_API_KEY")
except ImportError:
    # Try loading from .env file first, then fall back to environment
    try:
        from dotenv import load_dotenv
        load_dotenv()
    except ImportError:
        pass  # dotenv not available, will use os.getenv directly
    
    # Get from environment (either from .env or shell environment)
    GROQ_API_KEY = os.getenv("GROQ_API_KEY")
    BROWSER_USE_API_KEY = os.getenv("BROWSER_USE_API_KEY")

# Check if API keys are set
if not GROQ_API_KEY:
    print("Please set your Groq API key")
elif not BROWSER_USE_API_KEY:
    print("Please set your Browser Use API key")
else:
    print("Groq API key and Browser Use API key configured successfully!")

# Model configuration
MODEL = "openai/gpt-oss-120b"

Groq API key and Browser Use API key configured successfully!


In [16]:
def browser_research(query):
    """
    Research products using Groq + Browser Use MCP

    This function demonstrates the speed and accuracy of combining:
    - Groq's fast LLM inference (500+ tokens/second)
    - Browser Use MCP server for real-time web browsing
    """

    if not GROQ_API_KEY:
        print("Please set your Groq API key first!")
        return

    client = Groq(api_key=GROQ_API_KEY)

    print(f"{query}")

    start_time = time.time()

    # Configure MCP tools using Browser Use server
    tools = [
        {
            "type": "mcp",
            "server_url": "https://api.browser-use.com/mcp/",
            "server_label": "browseruse",
            "headers": {
                "X-Browser-Use-API-Key": BROWSER_USE_API_KEY
            }
        }
    ]

    try:
        # Call Groq with Browser Use MCP integration - use streaming to avoid timeouts
        completion = client.chat.completions.create(
            model=MODEL,
            messages=[
                {
                    "role": "system",
                    "content": "You are a helpful research assistant. Use browser tools to find accurate, up-to-date information. Keep tasks focused and simple - check one site at a time if needed."
                },
                {
                    "role": "user",
                    "content": query
                }
            ],
            temperature=0.3,
            max_completion_tokens=4096,
            top_p=0.8,
            stream=True,  # Use streaming to avoid timeouts
            tools=tools
        )

        print("=" * 80)
        print("BROWSER RESEARCH RESULTS")
        print("=" * 80)

        # Stream the response to avoid timeouts
        full_response = ""
        for chunk in completion:
            if chunk.choices[0].delta.content:
                content = chunk.choices[0].delta.content
                print(content, end="")
                full_response += content

        total_time = time.time() - start_time
        print("\\n" + "=" * 80)

        # Performance summary
        print("PERFORMANCE SUMMARY")
        print(f"   Total time: {total_time:.2f} seconds")
        print(f"   Response length: {len(full_response)} characters")
        print("Note: Response time includes real-time web browsing and data extraction across multiple sites - this is normal for autonomous web agent interactions!")

        return {
            "content": full_response,
            "total_time": total_time,
            "timestamp": datetime.now().isoformat(),
        }

    except Exception as e:
        total_time = time.time() - start_time
        error_msg = f"Browser research failed after {total_time:.2f} seconds: {str(e)}"
        print(f"\\n❌ {error_msg}")
        
        return {
            "content": error_msg,
            "total_time": total_time,
            "timestamp": datetime.now().isoformat(),
            "error": True
        }

## Demo 1: iPhone Comparison Research

Let's start by comparing iPhone models to see the speed and accuracy in action.

**What to watch for:**
- **Speed**: Response time with Groq + Browser Use MCP
- **Real-time data**: Current pricing and availability
- **Multi-source**: Information from Apple, carriers, and retailers
- **Intelligent analysis**: AI-powered comparison and recommendations

In [17]:
# Compare iPhone models with current pricing and availability
result_iphone = browser_research(
    "Whats the current price of iphone 15 pro on amazon and best buy?"
)

Whats the current price of iphone 15 pro on amazon and best buy?
BROWSER RESEARCH RESULTS
**iPhone 15 Pro (128 GB) – Approximate U.S. pricing (as of September 2025)**  

| Retailer | Listed price (USD) | Notes |
|----------|-------------------|-------|
| **Amazon** | **$999** (often listed at the Apple‑set MSRP) – occasional “Deal of the Day” or “Lightning Deal” can bring it down to **$979–$989**. | Prices can fluctuate quickly; check the product page for any limited‑time promotions, bundle offers, or refurbished options. |
| **Best Buy** | **$999** (standard MSRP) – Best Buy frequently runs “Open‑Box” or “Clearance” specials that drop the price to **$979–$989**. | Best Buy also offers a “Free Shipping” incentive and the option to add AppleCare+ at checkout. |

### How to verify the current price yourself
1. **Amazon** – Go to [amazon.com](https://www.amazon.com) and search for **“iPhone 15 Pro 128 GB”**. The price shown on the main product card (or the “Buy New” button) is the current

## Demo 2: Laptop Shopping Research

In [18]:
result_laptops = browser_research(
    "Find the top 3 laptops under $1000 by checking Amazon and Best Buy. Compare specs, prices, and customer reviews."
)

Find the top 3 laptops under $1000 by checking Amazon and Best Buy. Compare specs, prices, and customer reviews.
BROWSER RESEARCH RESULTS
**Top 3 Laptops under $1,000 (Amazon vs. Best Buy)**  

| Rank | Store | Model | Price* | Rating (★) | # Reviews | Processor | RAM | Storage | Screen | Notable Features |
|------|-------|-------|--------|------------|-----------|-----------|-----|---------|--------|-------------------|
| **1** | **Amazon** | **HP 15 Laptop** (incl. 1 yr Office 365) | **$349.99** (typical $379.99) | **4.8** | 249 | – (likely Intel i5/i7, not listed) | **16 GB** LPDDR4X | **1 TB** SSD | **15.6″** Full‑HD IPS | Office 365 bundle, back‑lit keyboard, fingerprint reader |
| **2** | **Amazon** | **Samsung Galaxy Book4** | **$759.00** | **4.9** | 36 | **Intel Core i7‑150U** (10‑core, 12‑thread, 1.2‑5.4 GHz) | **16 GB** LPDDR4X (6000 MHz) | **1 TB** NVMe SSD | **15.6″** Full‑HD LED | Windows 11 Pro, integrated Intel graphics, up to 15 h battery |
| **3** | **Amazon** | **NIMO


You just witnessed MCP in action - where models autonomously browse websites and extract information during generation. The model doesn't just generate text; it actively navigates web pages, compares prices, and analyzes product information across multiple sources.

The Browser Use integration here demonstrates autonomous web browsing. MCP works with any tools - APIs, databases, file systems. Ready to build AI that can research and act on the web? We are too.